# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0945,  0.0194,  0.0659,  0.0758,  0.1044,  0.1211, -0.1751,  0.0076,
          0.1757,  0.0199],
        [ 0.0761,  0.1271,  0.0391,  0.1010,  0.1327, -0.0531, -0.3762, -0.0177,
          0.1214,  0.0672]], grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[-0.1471, -0.1279,  0.0527,  0.0447,  0.1711, -0.2359,  0.1419,  0.2663,
         -0.0082,  0.0789],
        [-0.1429, -0.0686, -0.0329, -0.0810,  0.1319, -0.1076,  0.1784,  0.3255,
         -0.1224,  0.1617]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1170,  0.0824, -0.1410, -0.1406,  0.2379,  0.5042, -0.1280,  0.2716,
         -0.0592,  0.0527],
        [-0.1676,  0.0488, -0.2063,  0.0623,  0.2975,  0.3707, -0.2327,  0.2980,
         -0.1271, -0.0454]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0659, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2437, grad_fn=<SumBackward0>)